Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

In [2]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/dataset1/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/dataset2/h-or-s")
zip_ref.close()

In [3]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.999):
                print("\nCancelling training cus Accuracy reached 99.9 %")
                self.model.stop_training = True

    callbacks = myCallback()
    
    # Define and Compile the Model. assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])
        
    # create an instance of an ImageDataGenerator called train_datagen and a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/dataset2/h-or-s',            # This is the source directory for training images
        target_size=(150, 150),        # All images will be resized to 150x150
        batch_size=40,
        class_mode='binary')           # Since we use binary_crossentropy loss, we need binary labels

    # call model.fit_generator and train for a number of epochs
    history = model.fit_generator(train_generator,
      steps_per_epoch=8,  
      epochs=25,
      verbose=1, callbacks=[callbacks])
    return history.history['acc'][-1]

In [5]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/25
8/8 [==============================] - 4s 465ms/step - loss: 1.7404 - acc: 0.5500
Epoch 2/25
8/8 [==============================] - 3s 405ms/step - loss: 0.5577 - acc: 0.6781
Epoch 3/25
8/8 [==============================] - 3s 432ms/step - loss: 0.2201 - acc: 0.9312
Epoch 4/25
8/8 [==============================] - 3s 418ms/step - loss: 0.1077 - acc: 0.9688
Epoch 5/25
8/8 [==============================] - 3s 430ms/step - loss: 0.0647 - acc: 0.9781
Epoch 6/25
8/8 [==============================] - 3s 423ms/step - loss: 0.0897 - acc: 0.9625
Epoch 7/25
8/8 [==============================] - 3s 403ms/step - loss: 0.0683 - acc: 0.9812
Epoch 8/25
7/8 [=========================>....] - ETA: 0s - loss: 0.0057 - acc: 1.0000
Cancelling training cus Accuracy reached 99.9 %
8/8 [==============================] - 3s 377ms/step - loss: 0.0060 - acc: 1.0000


1.0